# 2. Perform sentiment analysis

In [1]:
import os
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy import StreamListener
import re
import json
#from HTMLParser import HTMLParser
from html.parser import HTMLParser
from textblob import TextBlob
#from keys_access_tokens import consumer_key, consumer_secret, access_token, access_secret
# Save your own keys and access tokens in "keys_access_tokens.py" acquired from Twitter Apps (https://apps.twitter.com/)


## 1. Loading the models

In [2]:
import pickle
from nltk.classify import ClassifierI
from statistics import mode
from nltk.tokenize import word_tokenize

In [3]:
class VoteClassifier(ClassifierI):
    def __init__(self, *classifiers):
        self._classifers = classifiers
        
    def classify(self, features):
        votes = []
        for c in self._classifers:
            v = c.classify(features)
            votes.append(v)
        return mode(votes)
        
    def confidence(self, features):
        votes = []
        for c in self._classifers:
            v = c.classify(features)
            votes.append(v)
        
        choice_votes = votes.count(mode(votes))
        confidence_factor = choice_votes / len(votes)

        return confidence_factor

In [4]:
def find_features(document):
    words = word_tokenize(document)
    features = {}
    for w in word_features:
        features[w] = (w in words)
    return features

In [5]:
def sentiment(text):
    features = find_features(text)
    return voted_classifier.classify(features), voted_classifier.confidence(features)

In [6]:
documents_file = open("pickles/documents.pickle", "rb") 
documents = pickle.load(documents_file)
documents_file.close()

word_features_file = open("pickles/word_features.pickle", "rb")
word_features = pickle.load(word_features_file)
word_features_file.close()


file = open("pickles/Naive_Bayes.pickle", "rb") 
NB_classifier = pickle.load(file)
file.close()

file = open("pickles/Multinomial_NB.pickle", "rb") 
MNB_classifier = pickle.load(file)
file.close()

file = open("pickles/Bernoulli_NB.pickle", "rb") 
BernoulliNB_classifier = pickle.load(file)
file.close()

file = open("pickles/Logistic_Regression.pickle", "rb") 
LogisticRegression_classifier = pickle.load(file)
file.close()

file = open("pickles/SGD_Classifier.pickle", "rb") 
SGD_Classifier = pickle.load(file)
file.close()

file = open("pickles/Linear_SVC.pickle", "rb") 
LinearSVC_classifier = pickle.load(file)
file.close()

file = open("pickles/Nu_SVC.pickle", "rb") 
NuSVC_classifier = pickle.load(file)
file.close()          

In [7]:
voted_classifier = VoteClassifier(NB_classifier, MNB_classifier, BernoulliNB_classifier,
                                  LogisticRegression_classifier, LinearSVC_classifier)

## Module for party data

In [8]:
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import os
import MySQLdb
import json
import re
import string
import time
from unidecode import unidecode
import csv

ModuleNotFoundError: No module named 'MySQLdb'

In [ ]:
#consumer key, consumer secret, access token, access secret which is passed to the oauth for tweepy.
consumer_key="Hg9dhbMGPQ3s2z5Jzde5oRwKu"
consumer_secret="iiJUk1zuWhUZdv4RICKuYxAqwVAVQ4MR3mX83RgPgYXcoFVUlF"
access_token="1068606021185282050-ZBPQSAo87KwUztDU8eLoCeTHS3h8zn"
access_secret="Yr7Sl41bmAZS9CRof4LZyHhUxLgUMflvVfepnC2FtpePq"

In [ ]:
# create mySQL connection to the local host using MySQLdb module.
# We are setting the charset to utf8mb4 to deal with smileys, emoticons, foriegn characters etc
conn = MySQLdb.connect("localhost","root","mongo1234","smdm",use_unicode=True, charset="utf8mb4")

c = conn.cursor()

In [ ]:
# this is our SQL for adding the tweetID and results into our DB
# always use %s no matter the type

add_tweet = ("INSERT INTO party_sentiment"
             "(tweetID, party_name, dateTime, tweet,screen_name,followers_count,friends_count,\
             verified_bit, source,country, country_code, full_name, name, place_type,\
             reply_count, retweet_count, favorite_count, sentiment, confidence,num_sentiment)"
             "VALUES ( %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)")

In [ ]:
def clean_tweet(tweet):
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

# Twitter's results just give us back the tweet, but don't tell us which keyword it was found with
# so, we have to use a keyword dictionary to search the tweet and match it back up to the party
party_tags = dict()
#BJP tags
party_tags['modi'] = 'Bharatiya Janata Party'
party_tags['namo'] = 'Bharatiya Janata Party'
party_tags['namo'] = 'Bharatiya Janata Party'
party_tags['phirekbaarmodisarkar'] = 'Bharatiya Janata Party'
party_tags['bharatiyajanataparty']  = 'Bharatiya Janata Party'
party_tags['bjp'] = 'Bharatiya Janata Party'
party_tags['nationaldemocraticalliance'] = 'Bharatiya Janata Party'
party_tags['nda'] = 'Bharatiya Janata Party'
party_tags['vajpayee'] = 'Bharatiya Janata Party'
#congress tags
party_tags['indiannationalcongress'] = 'Indian National Congress'
party_tags['incindia'] = 'Indian National Congress'
party_tags['gandhi'] = 'Indian National Congress'
party_tags['rahulgandhi'] = 'Indian National Congress'
party_tags['soniagandhi'] = 'Indian National Congress'
party_tags['sonia'] = 'Indian National Congress'
party_tags['priyankagandhi'] = 'Indian National Congress'
party_tags['rahulgandhiforpm'] = 'Indian National Congress'
party_tags['inc'] = 'Indian National Congress'
party_tags['congress'] = 'Indian National Congress'
party_tags['amethi'] = 'Indian National Congress'



#exclude punctuations
regex = re.compile('[%s]' % re.escape(string.punctuation))

# Open/create a file to append data to
csvFile = open('crawl_result2.csv', 'a')

#Use csv writer
csvWriter = csv.writer(csvFile)

#build the class used to process tweets to check for feeds
class twitter_streaming(StreamListener):

    def on_data(self, data):
        all_data = json.loads(HTMLParser().unescape(data))
        #https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/tweet-object
        #https://gist.github.com/hrp/900964
        if 'text' in all_data:
            #1
            tweet = all_data['text']
            tweet = unidecode(tweet)
            #2
            tweetID = all_data['id_str']
            #3
            source = all_data['source']
            source = unidecode(source)
            #4
            if all_data['place']:
                country = all_data['place']['country']
                country = unidecode(country)
                #5
                country_code = all_data['place']['country_code']
                country_code = unidecode(country_code)
                #6
                full_name = all_data['place']['full_name']
                full_name = unidecode(full_name)
                #7
                name =  all_data['place']['name']
                name = unidecode(name)
                #8
                place_type = all_data['place']['place_type']
                place_type = unidecode(place_type)
                #9
            else:
                country = country_code = full_name = name = place_type = "0"
            
            quote_count = all_data['quote_count']
            #10
            reply_count = all_data['reply_count']
            #11
            retweet_count = all_data['retweet_count']
            #12
            favorite_count = all_data['favorite_count']
            #13
            screen_name = all_data['user']['screen_name']
            screen_name = unidecode(screen_name)
            #13
            followers_count = all_data['user']['followers_count']
            #14
            friends_count = all_data['user']['friends_count']
            #15
            verified = all_data['user']['verified']
            #print("verified value is:", verified)
            #type(verified)
        
            
 
            #tweetNoPunctuation = regex.sub('', tweet)
            tweetNoPunctuation = clean_tweet(tweet)
#we want to make sure while compiling tweets, we do not include the oens that are retweeted
            if not all_data['retweeted'] and not tweet.startswith('RT') and 't.co' not in tweet:
                sentiment_value, confidence = sentiment(tweetNoPunctuation)
                #print(tweet, sentiment_value, confidence) #print output
                
                #value manipulations
                if (sentiment_value.lower() == "neg"):
                    num_sentiment=0
                else:
                    num_sentiment=1
                    
                if (verified == True):
                    verified_bit = 1
                    #print("Set")
                else:
                    verified_bit = 0

                
                    
                found = False
                party = ""
                for word in tweetNoPunctuation.split(" "):  
                    if word.lower() in party_tags.keys():
                        party_name = party_tags[word.lower()]
                        print("Found keyword: ", word, " belongs to party: ", party_name)
                        found = True
                        break

                if found:
                    created_at = time.strftime('%Y-%m-%d %H:%M:%S')  
                    newID = (int)(all_data['id'])
                    #twitter JSON is being parsed with queries below and using sentiment module, we are assigning confidence values
                    # tweetID, party_name, dateTime, tweet, source,country, country_code, full_name, name, place_type,\
                    #  reply_count, retweet_count, favorite_count, result, confidence,num_sentiment
                    tweet_data = (tweetID ,party_name, created_at, tweet,screen_name,followers_count,friends_count,\
                                  verified_bit, source,country, country_code,full_name,name, place_type,\
                                  reply_count, retweet_count, favorite_count, sentiment_value.lower(), confidence, num_sentiment)
                    
                    # Write a row to the CSV file. I use encode UTF-8
                    csvWriter.writerow([tweetID ,party_name, created_at, tweet,screen_name,followers_count,friends_count,\
                                        verified_bit, source,country, country_code,full_name,name, place_type,\
                                  reply_count, retweet_count, favorite_count, sentiment_value.lower(), confidence, num_sentiment])
                    
                    c.execute(add_tweet, tweet_data)
                    conn.commit()
                    
        #error handling, since tweepy tends to time out with twitter with out any reason closing the connection from their side
        def on_limit(self, track):
            print('Limit hit! Track = %s' % track)
            return True

        def on_error(self, status):
            print(status)

        def on_disconnect(self, notice):
            print(notice)

        return True

In [ ]:
# Twitter Authorization path
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

while True:
    try:
        # create twitter stream, which in turn will start streaming tweeets in JSON format, 
        #which we are using to query its metadata and store them separately on to the database
        twitterStream = Stream(auth, twitter_streaming())

        party keywords  to search for in tweets
        twitterStream.filter(track=["modi","namo","narendramodi","phirekbaarmodisarkar", "bharatiyajanataparty","bjp",\
                                     "nationaldemocraticalliance", "nda",\
                                     "vajpayee", "indiannationalcongress", "gandhi", "rahulgandhi", "soniagandhi","priyankagandhi",\
                                     "rahulgandhiforpm"], languages=["en"])
        
       # twitterStream.filter(track=["amethi", "incindia", "indiannationalcongress","sonia", "gandhi", "rahulgandhi", "soniagandhi","priyankagandhi",\
                                    "rahulgandhiforpm"], languages=["en"])
    except:
        continue

cursor.close()
conn.close()
csvFile.close()

https://www.dataquest.io/blog/matplotlib-tutorial/

https://ritetag.com/best-hashtags-for/bjp

### Another

In [ ]:
out1 = "twitter-feed.txt"
os.remove(out1) if os.path.exists(out1) else None

out2 = "twitter-feed-textblob.txt"
os.remove(out2) if os.path.exists(out2) else None

positive, negative, neutral = 0.0, 0.0, 0.0

    
class listener(StreamListener):    
    def on_data(self, data):


        all_data = json.loads(data)
        tweet = clean_tweet(all_data["text"])
        print (tweet)
        
        category, confidence = sentiment(tweet)  # Custom-sentiment
        print ("Custom Sentiment : ", category, "-->", confidence)


        if confidence * 100 >= 80:
            output = open(out1, "a")
            output.write(category)
            output.write("\n")
            output.close()
        
        blob = TextBlob(tweet)
        confidence = blob.sentiment.polarity  # Textblob-sentiment
        if confidence > 0:
            category = "pos"
#             positive += 1.0
        else:
            category = "neg" 
#             negative += 1.0

        
        print ("TextBlob Sentiment : ", category, "-->", "{:0.2f}" .format(confidence))
        print ()
        
        output = open(out2, "a")
        output.write(category)
        output.write("\n")
        output.close()
            
        return True
    
    def on_error(self, status):
        print (status)     
        
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

query = input("Enter your keyword to be searched in Twitter: ")

output = open(out1, "a")
output.write(query)
output.write("\n")
output.close()

output = open(out2, "a")
output.write(query)
output.write("\n")
output.close()

twitterStream = Stream(auth, listener())
twitterStream.filter(track=[query])

In [ ]:
import html
import os
import plotly
import socket

from twython import Twython
from twython import TwythonAuthError, TwythonError, TwythonRateLimitError

def chart(positive, negative):
    """Return a pie chart for specified sentiments as HTML."""

    # offline plot
    # https://plot.ly/python/pie-charts/
    # https://plot.ly/python/reference/#pie
    figure = {
        "data": [
            {
                "labels": ["positive", "negative"],
                "hoverinfo": "none",
                "marker": {
                    "colors": [
                        "rgb(0,255,00)",
                        "rgb(255,0,0)"
                    ]
                },
                "type": "pie",
                "values": [positive, negative]
            }
        ],
        "layout": {
            "showlegend": True
            }
    }
    return plotly.offline.plot(figure, output_type="div", show_link=False, link_text=False)
# generate chart
chart = chart(positive, negative)

### SQL Queries

use smdm;
#Table, Create Table
 CREATE TABLE `party_sentiment` (
 `id_t1` bigint NOT NULL AUTO_INCREMENT,
  `tweetID` varchar(200) CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci NOT NULL,
  `party_name` varchar(200) CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci NOT NULL,
  `dateTime` datetime(6) DEFAULT NULL,
  `tweet` varchar(200) CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci NOT NULL,
  `source` varchar(200) CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci NOT NULL,
  `country` varchar(200) CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci,
  `country_code` varchar(200) CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci,
  `full_name` varchar(200) CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci,
  `name` varchar(200) CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci,
  `place_type` varchar(200) CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci,
  `quote_count` int(11) DEFAULT NULL,
  `reply_count` int(11) DEFAULT NULL,
  `retweet_count` int(11) DEFAULT NULL,
  `favorite_count` int(11) DEFAULT NULL,
  `result` varchar(6) DEFAULT NULL,
  `confidence` float DEFAULT NULL,
  `num_sentiment` int(11) DEFAULT NULL,
  primary key (id_t1)
)ENGINE=InnoDB DEFAULT CHARSET=utf8;

#delete all data from table
DELETE FROM party_sentiment;

#seed auto increment
ALTER TABLE party_sentiment AUTO_INCREMENT = 1;

